In [64]:
import import_ipynb
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import bertkeras as bk
import BERTDictionary as bd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
from unidecode import unidecode
from scipy.stats import norm, boxcox
pd.options.display.max_colwidth = 1000
sns.set()

In [65]:
df= pd.read_csv('../dataset/training.csv', index_col='ID')
labels = df['label']
df = df.drop(columns=[ '﻿ASIN','parent_asin','Product Group Description'])
df = df.fillna("")

cols = df.columns.tolist()
cols.remove('label')
text = df[cols].apply(' '.join, axis=1)


In [66]:
html_re = re.compile('<[^>]*>')
garbage_re = re.compile('[^0-9a-z\'\.]+') 
weight_re = re.compile('[\d.]+[ ]?(kg+|lbs+|g+|lb+)')

# check if it actually helps, tests https://regex101.com/r/xwnV5l/1
size_re = re.compile('[\d.]+([ cminch]+)?[ x*]+[\d.]+([ cminch]+)?([ x*]+[\d.]+)?([ cminch]+)')

def html_cleanup(html):
    return re.sub(html_re, ' ', html)
def lower_case(txt):
    return txt.lower()
def split_by_garbage(txt):
    return re.sub(garbage_re, ' ', txt)
def decode_chars(txt):
    return unidecode(txt)

def cleanup_size(txt):
    return re.sub(size_re, ' ', txt).replace('size:', '')

def cleanup_weight(txt):
    return re.sub(weight_re, ' ', txt)
    
def cleanup(text):
    res = text.apply(html_cleanup).apply(decode_chars).apply(lower_case) \
              .apply(split_by_garbage).apply(cleanup_size) \
              .apply(cleanup_weight)
    return res

In [67]:
texts = cleanup(text)

In [68]:
df = pd.DataFrame( {"texts" :texts, "labels": labels} )

In [69]:
# there is error in training dataset
df.loc[196,'labels'] =1


In [70]:
txt_train, txt_test, lbl_train, lbl_test = train_test_split(
    texts, labels, test_size=0.001, random_state=42)

In [71]:
lbl_train.tail()

ID
398     0
5823    0
5943    0
3822    1
6054    0
Name: label, dtype: int64

In [72]:
k=10000
w = ' '.join(bd.adult_toys_vocab)
txt_train = txt_train.append(pd.Series([w], index=[k]))
lbl_train = lbl_train.append(pd.Series([1], index=[k]))
w

'[NEL] porn nubby lesbian gay transvestite sex intercourse fuck vibrate suction lick suck dildo vibrator condom anal plug trembler pulsator lockaid strap buzzer rabbit egg penetrator dual kegel ben wa stimulator masturbator lasso bondage cuff pasties device gadget pleasure intimate last aroused climax enjoyment orgasm bdsm sadism submission masochism realistic stimulate dick cock penis spot butt ass nipple clitor pelvic vagina pussy anus prostate rectum ventral scrotum'

In [73]:
tokenizer = bk.create_tokenizer_from_hub_module(vocab_file='vocab.txt')
max_seq_length = 512

using vocab file from vocab.txt


In [74]:
# Convert data to InputExample format
train_examples = bk.convert_text_to_examples(txt_train, lbl_train.values)
test_examples = bk.convert_text_to_examples(txt_test, lbl_test.values)

In [75]:

# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = bk.convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = bk.convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

In [76]:
train_examples[0].__dict__

{'guid': None,
 'text_a': 'round emerald 14k rose gold engagement ring aeur rosemary ring a classic basket setting holds an 7mm round cut center stone flanked by two 5mm round cut accent stones in this sparkling ring carefully handcrafted in your choice of gems and metals. other products ',
 'text_b': None,
 'label': 0}

In [77]:
def build_model(max_seq_length): 
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = bk.BertLayer(n_fine_tune_layers=1, pooling="first")(bert_inputs)
    dense = tf.keras.layers.Dense(256, activation='relu')(bert_output)
    pred = tf.keras.layers.Dense(1, activation='sigmoid')(dense)
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [78]:
ACCURACY_THRESHOLD = 0.999
LOSS_THRESHOLD = 0.04
class myCallback(tf.keras.callbacks.Callback): 
    def on_batch_end(self, batch, logs): 
        if(logs.get('acc') > ACCURACY_THRESHOLD and logs.get('loss')<LOSS_THRESHOLD):   
          print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100)) 
          self.model.stop_training = True

In [79]:
model = build_model(max_seq_length)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          (None, 512)          0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
bert_layer_2 (BertLayer)        (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]                
          

In [80]:
# Initialize session
sess = tf.Session()
# Instantiate variables
bk.initialize_vars(sess)
tb = tf.compat.v1.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=24, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='batch')
stop999 = myCallback()
model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    callbacks = [ tb], #, stop999
    epochs=2,
    batch_size=24
)

Train on 5481 samples, validate on 6 samples
Epoch 1/2
5481/5481 [==============================] - 241s 44ms/sample - loss: 0.1310 - acc: 0.9586 - val_loss: 0.0713 - val_acc: 1.0000
Epoch 2/2
5481/5481 [==============================] - 236s 43ms/sample - loss: 0.0798 - acc: 0.9799 - val_loss: 0.0331 - val_acc: 1.0000


In [81]:
model.save('restricted.h5')

In [82]:
df_subm= pd.read_csv('../dataset/public_test_features.csv', index_col='ID')
df_subm = df_subm.drop(columns=[ '﻿ASIN','parent_asin','Product Group Description'])
df_subm = df_subm.fillna("")

text_subm = df_subm[cols].apply(' '.join, axis=1)


In [83]:
texts_subm = cleanup(text_subm)

In [84]:
subm_examples = bk.convert_text_to_examples(texts_subm, [0]*len(texts_subm))

In [85]:
(subm_input_ids, subm_input_masks, subm_segment_ids, subm_labels
) = bk.convert_examples_to_features(tokenizer, subm_examples, max_seq_length=max_seq_length)

In [86]:
subm_examples[0].text_a

'vintage mama e hija amor corazon cadena collar mujeres dia de la madre regalo especificaciones disea o de mama e hija colgante con forma de coraza3n. hecho de material de aleacia3n de alta calidad con mano de obra fina. regalo perfecto para el daa de la madre u otras ocasiones. tipo collar ganero mujeres tema estilo vintage material aleacia3n de ocasiones fiesta daa de la madre regalo vida diaria etc. caracterasticas colgante de coraza3n patra3n de letras regalo para madre longitud de la cadena  69 pulgadas  pulgadas aprox. notas debido a la diferencia de configuracia3n de la luz y la pantalla el color del artaculo puede ser ligeramente diferente al de las ima genes. puede haber una ligera diferencia de dimensia3n debido a las diferentes medidas manuales. el paquete incluye 1 x collar item specifics fino o de moda moda tipo de articulo collares nombre de la marca bluelans estilo trendy forma patron carta tipo de cadena cadena de eslabones tamano colgante 5cm compatibilidad totalmente 

In [87]:
testCount = len(subm_input_ids)
def pickMax(arr):
  m = max(arr)
  return [x for x in arr]

threshold = 0.5
def tolabel(arr):
  # return int(max(arr)>=threshold)
  return max(arr)
    


In [ ]:
predictions = model.predict([subm_input_ids[0:testCount], 
                                subm_input_masks[0:testCount], 
                                subm_segment_ids[0:testCount]])

In [ ]:
pred = [ tolabel(p) for p in predictions]

In [ ]:
results = pd.DataFrame({"text":texts_subm, "label":pred})

In [ ]:
results[results['label']>0.05][results['label']<0.9]


In [ ]:
results.to_csv('results.csv', columns=[ 'label'])

In [ ]:
bd.adult_toys_vocab